# Build a Custom SciKit Learn Model

In this notebook we will demonstrate what is required to train and deploy a custom model on Sagemaker infrastructure. 


## Create a training script

We need to define a model that can run on Sagemaker training hardware.

Detailed guidance here https://sagemaker.readthedocs.io/en/stable/using_sklearn.html#preparing-the-scikit-learn-training-script


In [1]:
%%writefile sklearn_training_script.py

import argparse
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ =='__main__':
    
    #------------------------------- parsing input parameters (from command line)
    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # RandomForest hyperparameters
    parser.add_argument('--n_estimators', type=int, default=10)
    parser.add_argument('--min_samples_leaf', type=int, default=3)
    
    # Data, model, and output directories
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train_dir', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test_dir', type=str, default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train_file', type=str, default='train.csv')
    parser.add_argument('--test_file', type=str, default='validation.csv')
    parser.add_argument('--features', type=str, default='')  # explicitly name which features to use
    parser.add_argument('--target_variable', type=str)  # explicitly name the column to be used as target

    args, _ = parser.parse_known_args()
    
    #------------------------------- data preparation
    print('reading data')
    train_df = pd.read_csv(os.path.join(args.train_dir, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test_dir, args.test_file))

    features = args.features.split()
    if features == []:
        features = list(train_df.columns)
        features.remove(args.target_variable)
    
    print('building training and testing datasets')
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[args.target_variable]
    y_test = test_df[args.target_variable]
    
    #------------------------------- model training
    print('training model')
    model = RandomForestClassifier(
        n_estimators=args.n_estimators,
        min_samples_leaf=args.min_samples_leaf,
        n_jobs=-1)
    
    model.fit(X_train, y_train)
    
    #-------------------------------  model testing
    print('testing model')
    #abs_err = np.abs(model.predict_proba(X_test) - y_test)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test))
    print("Validation AUC: ", roc_auc)
        
    #------------------------------- save model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print('model saved at ' + path)

Writing sklearn_training_script.py



#### Local training

Script arguments allows us to remove from the script any SageMaker-specific configuration, and run locally

**Note** This script relies on scikit-learn version 0.22 (Certain functions have been deprecated in 0.23)


In [ ]:
! python sklearn_training_script.py \
    --n_estimators 100 \
    --min_samples_leaf 3 \
    --model_dir 'model/' \
    --train_dir '../../data/partitioned/' \
    --test_dir '../../data/partitioned/' \
    --train_file 'train.csv' \
    --test_file 'validation.csv' \
    --target_variable 'readmitted'


## Create a script to execute training

To build the model we execute on Sagemake infrastructure using the SKLearn Estimator.
https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html

Write out a script for this (that can be run locally or via the master RUN script)

In [3]:
%%writefile RUN_Sagemaker_Build.py

# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
import sagemaker
import sys

sys.path.append("../../")
import utils.config as cfg
import utils.models as mods

config = cfg.get_config()
region = config['region']
target = config['target']
bucket_name = config['bucket_name']
bucket_prefix = config['bucket_prefix']
sgmk_session = config['sgmk_session']
sgmk_role = config['sgmk_role']
sm_boto3 = config['sm_boto3']

train_path_s3 = cfg.get_s3_path('train')
test_path_s3 = cfg.get_s3_path('validation')

sklearn_estimator = SKLearn(
    entry_point='source/sklearn_training_script.py',
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version='0.20.0',
    base_job_name='rf-scikit',
    metric_definitions=[
        { 'Name': 'AUC', 'Regex': 'Validation AUC: ([0-9.]+).*$' },
    ],
    hyperparameters={
        'n_estimators': 100,
        'min_samples_leaf': 3,
        'target_variable': target,
    },
    max_run=20*60,  # Maximum allowed active runtime (in seconds)
    use_spot_instances=True,  # Use spot instances to reduce cost
    max_wait=30*60,  # Maximum clock time (including spot delays)
)

sklearn_estimator.fit({'train':train_path_s3, 'test': test_path_s3}, wait=True)

sklearn_estimator.latest_training_job.wait(logs='None')

model_artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)['ModelArtifacts']['S3ModelArtifacts']

print('Model artifact saved at:', model_artifact)

model = SKLearnModel(
    model_data=model_artifact,
    framework_version='0.20.0',
    py_version='py3',
    role=get_execution_role(),
    entry_point='sklearn_training_script.py',
)

predictor = model.deploy(
    instance_type='ml.c5.large',
    initial_instance_count=1,
)

mods.register("Custom_SKLearn_RF", "Random Forest Classifier with Added Features", model_artifact, predictor)

Writing RUN_Sagemaker_Build.py


### Execute it

In [ ]:
!python RUN_Sagemaker_Build.py